In [73]:
"""You can define global variables here """

from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib
import pandas as pd

np.set_printoptions(suppress=True,
   formatter={'float_kind':'{:16.3f}'.format}, linewidth=80)

global model # We want to use the same model throughout the interface
global company_num

def start( company, name ):
    """Called once at the beginning of testing period.
        Company is the number of the company you are predicting (0-2), or 3 if you are to
        predict the value of all companies.
        Name is the (path and) name of your model.
        You should load the model you use throughout here (no training to take place) """
    print("Starting model of company %d" % company)
    file_path = "models/" + name
    print("Loading model from \"" + file_path + "\"")
    from keras.models import load_model
    global model 
    global company_num
    company_num = company
    model = load_model(file_path)
    return


def predict( stock_prices, market_analysis, market_segments, info ):
    """Called on consecutive days of the test period (when stock markets are open).
       The arguments are provided as lists of tuples, ordered in an increasing
       order by date (i.e., the last tuple in stock_prices and info, are for the day you are
       predicting. You are not allowed to change the arguments.

       You are to predict the stock-price movement for the date indicated by the last record in
       the info list. Note, that the lists also contain information about older dates, but you are
       free to ignore them if not using them.

       Returns a 3-list of predictions (company 0, 1, 2, respectively). Each value is True if
       prediction is that stock price for respective company will go up, otherwise False
       (i.e. if you think company 0 will go up, but not 1 and 2, then return [True, False, False]).
       If you are only predicting a single company, set the respective field but keep the others False
       (i.e. if company is number 1 and you predict its stock will go up, then return [False, True, False]).
    """
    prediction = [False, False, False]
    
    
    # If it is a single company we run this...
    if company_num != 3:
        # Getting the info of today
        info_today = info[len(info) - 1]
    
        # Getting stock_price of today
        stock_today = stock_prices[len(stock_prices) - 1]
        stock_yesterday = stock_prices[len(stock_prices) - 2]
        
        # Get the info we use in our model
        X = info_today[4:]
        
        if company_num != 1:
            # IT = [1,0]
            X = np.hstack((X, [1, 0]))
        else:
            X = np.hstack((X, [0, 1]))
        
        # Make the stock price binary based on yesterday
        if stock_today[4] > stock_yesterday[4]:
            X = np.hstack((X, [1]))
        else:
            X = np.hstack((X, [0]))
        print("Should be: ", X[-1])
        
        # Load the scaler used to normalize the training data
        scaler = joblib.load("models/scaler Company " + str(company_num) + ".save")
        X = X.reshape(1, -1)
        X_test = scaler.transform(X)
        X_test = np.array(X_test[0][:-1])
        
        # Prediction time!
        predict = model.predict_classes(X_test.reshape(1,-1))
        print("Prediction: ", predict[0][0])
        print("-"*10)
        if predict[0][0] == 1:
            prediction[company_num] = True
        
    # Now for the combination!
    elif company_num == 3:
        # Company 0
        info_today_0 = info[len(info) - 3]
        X = info_today_0[4:]
        X = np.hstack((X, [1, 0]))
        stock_today_0 = stock_prices[len(stock_prices) - 3]
        stock_yesterday_0 = stock_prices[len(stock_prices) - 6]
        
        # Company 1
        info_today_1 = info[len(info) - 2][4:]
        X = np.hstack((X, info_today_1))
        X = np.hstack((X, [0, 1]))
        stock_today_1 = stock_prices[len(stock_prices) - 2]
        stock_yesterday_1 = stock_prices[len(stock_prices) - 5]
        
        # Company 2
        info_today_2 = info[len(info) - 1][4:]
        X = np.hstack((X, info_today_2))
        X = np.hstack((X, [0, 1]))
        stock_today_2 = stock_prices[len(stock_prices) - 1]
        stock_yesterday_2 = stock_prices[len(stock_prices) - 4]
        
        # Putting it all together
        all_info = [info_today_1,info_today_2]
        all_stock = [stock_today_0,stock_today_1,stock_today_2]
        all_stock_y = [stock_yesterday_0, stock_yesterday_1, stock_yesterday_2]
            
        for i in range(len(all_stock)):
            if all_stock[i][4] > all_stock_y[i][4]:
                X = np.hstack((X, [1]))
            else:
                X = np.hstack((X, [0]))
                
        scaler = joblib.load("models/scaler Company 13.save")
        X = X.reshape(1, -1)
        X_test = scaler.transform(X)
        X_test = np.array(X_test[0][:-3])
        
        predict = model.predict(X_test.reshape(1,-1))
        for prob in predict[0]:
            print(prob)
        #print("Prediction: ", predict)
        print("-"*10)
        
        
        
    return prediction

def end():
    """Called once at the end of the testing period. Optional if you do anything here."""
    ...
    return


def main():
    
    info_data = pd.read_csv("data/info.txt", sep='\s+')
    market_analysis = pd.read_csv("data/market_analysis.txt", sep='\s+')
    market_segments = pd.read_csv("data/market_segments.txt", sep='\s+')
    stock_prices = pd.read_csv("data/stock_prices.txt", sep='\s+')
    
    days_back = 100
    
    info_0 = info_data[info_data['company'] == 0]
    info_0 = info_0[-days_back:]
    info_0 = [tuple(x) for x in info_0.values]
    stock_prices_0 = stock_prices[stock_prices['company'] == 0]
    stock_prices_0 = stock_prices_0[-days_back:]
    stock_prices_0 = [tuple(x) for x in stock_prices_0.values]
    
    info_1 = info_data[info_data['company'] == 1]
    info_1 = info_1[-days_back:]
    info_1 = [tuple(x) for x in info_1.values]
    stock_prices_1 = stock_prices[stock_prices['company'] == 1]
    stock_prices_1 = stock_prices_1[-days_back:]
    stock_prices_1 = [tuple(x) for x in stock_prices_1.values]
    
    info_2 = info_data[info_data['company'] == 2]
    info_2 = info_2[-days_back:]
    info_2 = [tuple(x) for x in info_2.values]
    stock_prices_2 = stock_prices[stock_prices['company'] == 2]
    stock_prices_2 = stock_prices_2[-days_back:]
    stock_prices_2 = [tuple(x) for x in stock_prices_2.values]
    
    
    module = 3
    start(module, "c13.h5")
    daily_stock = []
    daily_info = []
    if module != 3:
        for i in range(0,days_back):
            day = -days_back + i
            daily_stock.append(stock_prices_0[day])
            daily_info.append(info_0[day])
            predict( daily_stock, 0, 0, daily_info)
    elif module == 3:
        for i in range(0,days_back):
            day = -days_back + i
            daily_stock.append(stock_prices_0[day])
            daily_info.append(info_0[day])
            daily_stock.append(stock_prices_1[day])
            daily_info.append(info_1[day])
            daily_stock.append(stock_prices_2[day])
            daily_info.append(info_2[day])
            predict( daily_stock, 0, 0, daily_info )

main()

Starting model of company 3
Loading model from "models/c13.h5"
8.80238e-05
0.0002846387
0.3792651
----------
0.03027179
0.36613026
0.0044989735
----------
0.7559162
0.009590855
0.75733215
----------
0.9666635
0.03790437
0.7235252
----------
0.94243073
8.0124904e-05
0.25688866
----------
1.8182594e-05
0.00051705085
0.7048131
----------
0.9988782
0.03619811
0.9778836
----------
0.015628552
0.009320173
0.00035151985
----------
0.00036689077
0.8173673
0.0022071502
----------
0.22656742
0.73611027
0.07578071
----------
0.0018745414
0.23659931
0.19011712
----------
0.006924806
0.0006765508
0.9973449
----------
0.7631427
0.0040688757
0.29446742
----------
0.5747336
0.87776333
5.8030917e-05
----------
0.9815517
0.75667334
0.0011020325
----------
0.5587208
0.00024908024
0.21478635
----------
0.5203659
0.6112572
0.024251234
----------
3.7056292e-05
0.011480312
0.7388572
----------
0.9802058
0.00831883
0.9893755
----------
0.0015593866
0.0014528331
0.3126282
----------
0.99888533
0.9978065
0.0533